In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
 
%matplotlib inline

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [ ]:
df_ = pd.read_csv("train.csv")
df_.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
#df_.drop('ID',axis='columns',inplace=True)

In [ ]:
df_.dtypes

ID                     object
Gender                 object
Age                     int64
Region_Code            object
Occupation             object
Channel_Code           object
Vintage                 int64
Credit_Product         object
Avg_Account_Balance     int64
Is_Active              object
Is_Lead                 int64
dtype: object

In [ ]:
df[df.isna().any(axis=1)].shape

(29325, 11)

In [ ]:
df_.shape

(245725, 11)

In [ ]:
Y = df['Is_Lead']
X = df.drop('Is_Lead',axis='columns')

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.1,random_state=1)

In [ ]:
X_train.shape

(221152, 10)

In [ ]:
Y_test.shape

(24573,)

In [ ]:
#X_train['Credit_Product'].fillna(X_train['Credit_Product'].mode()[0], inplace=True)
#X_train.head()

In [ ]:
#X_train.Credit_Product.replace(('Yes','No'),(1,0),inplace=True)

In [ ]:
def datapreprocess(X_dat):
  X_dat.drop('ID',axis='columns',inplace=True)
  X_dat.Credit_Product.replace(('Yes','No'),(1,0),inplace=True)
  X_dat.Is_Active.replace(('Yes','No'),(1,0),inplace=True)
  X_dat['Credit_Product'].fillna(X_dat['Credit_Product'].mode()[0], inplace=True)
  X_dat = pd.get_dummies(data=X_dat,columns=['Gender','Occupation','Channel_Code'])
  X_dat.drop('Region_Code',axis='columns',inplace=True)
  col_scale = ["Age","Vintage","Avg_Account_Balance"]
  scaler = MinMaxScaler()
  X_dat[col_scale] = scaler.fit_transform(X_dat[col_scale])
  return X_dat


In [ ]:
X_train_prep = datapreprocess(X_train)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
X_train_prep.sample(10)

,Age,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Gender_Female,Gender_Male,Occupation_Entrepreneur,Occupation_Other,Occupation_Salaried,Occupation_Self_Employed,Channel_Code_X1,Channel_Code_X2,Channel_Code_X3,Channel_Code_X4
75337,0.080645,0.093750,1.0,0.032233,0,0,1,0,0,1,0,1,0,0,0
144756,0.048387,0.062500,0.0,0.072939,0,1,0,0,0,1,0,1,0,0,0
80801,0.177419,0.148438,0.0,0.096279,0,0,1,0,1,0,0,1,0,0,0
206307,0.064516,0.054688,1.0,0.146036,0,1,0,0,0,1,0,1,0,0,0
236067,0.354839,0.515625,1.0,0.070748,1,0,1,0,0,0,1,0,1,0,0
172865,0.548387,0.296875,1.0,0.084338,0,1,0,0,0,0,1,0,0,1,0
206076,0.048387,0.109375,0.0,0.126458,0,1,0,0,0,1,0,1,0,0,0
20981,0.500000,0.328125,1.0,0.057272,0,0,1,0,0,0,1,0,1,0,0
102513,0.064516,0.046875,0.0,0.079761,0,0,1,0,0,1,0,1,0,0,0
220661,0.370968,0.523438,0.0,0.105511,0,0,1,0,0,0,1,0,1,0,0


In [ ]:
X_train_prep.describe()

,Age,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Gender_Female,Gender_Male,Occupation_Entrepreneur,Occupation_Other,Occupation_Salaried,Occupation_Self_Employed,Channel_Code_X1,Channel_Code_X2,Channel_Code_X3,Channel_Code_X4
count,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000,221152.000000
mean,0.336466,0.311832,0.293070,0.107161,0.388710,0.454163,0.545837,0.010902,0.285962,0.293074,0.410062,0.421592,0.275950,0.279663,0.022794
std,0.239287,0.252607,0.455171,0.082347,0.487458,0.497896,0.497896,0.103842,0.451872,0.455173,0.491846,0.493815,0.446993,0.448835,0.149247
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.112903,0.101562,0.000000,0.056500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.322581,0.195312,0.000000,0.084602,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.500000,0.515625,1.000000,0.130302,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
X_train_prep.shape

(221152, 15)

In [ ]:
model1 = Sequential([
                          
                          Dense(16,input_shape=(15,),kernel_regularizer=regularizers.l2(0.001),activation='relu'),
                          Dropout(0.2),
                          Dense(8,kernel_regularizer=regularizers.l2(0.001),activation='relu'),
                          Dropout(0.2),
                          Dense(1,activation='sigmoid')
                          
])

In [ ]:
early_stopping = EarlyStopping(monitor='val_prc',patience=30)
#learning_rate_reduction = ReduceLROnPlateau(factor=0.2,patience=15)

In [ ]:
from tensorflow import keras

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


In [ ]:
model1.compile(optimizer='adam',
                  loss='binary_crossentropy',metrics=METRICS)

In [ ]:
history = model1.fit(X_train_prep, Y_train, epochs=100, batch_size=256, validation_split=0.12,
                         callbacks=[early_stopping,tf.keras.callbacks.CSVLogger("results.csv")])

Epoch 1/100
761/761 [==============================] - 6s 4ms/step - loss: 0.5273 - tp: 3135.0000 - fp: 3264.0000 - tn: 145168.0000 - fn: 43046.0000 - accuracy: 0.7620 - precision: 0.4899 - recall: 0.0679 - auc: 0.6830 - prc: 0.3747 - val_loss: 0.4862 - val_tp: 600.0000 - val_fp: 142.0000 - val_tn: 20119.0000 - val_fn: 5678.0000 - val_accuracy: 0.7807 - val_precision: 0.8086 - val_recall: 0.0956 - val_auc: 0.7493 - val_prc: 0.4863
Epoch 2/100
761/761 [==============================] - 2s 3ms/step - loss: 0.4920 - tp: 4991.0000 - fp: 2240.0000 - tn: 146192.0000 - fn: 41190.0000 - accuracy: 0.7768 - precision: 0.6902 - recall: 0.1081 - auc: 0.7348 - prc: 0.4720 - val_loss: 0.4757 - val_tp: 739.0000 - val_fp: 205.0000 - val_tn: 20056.0000 - val_fn: 5539.0000 - val_accuracy: 0.7836 - val_precision: 0.7828 - val_recall: 0.1177 - val_auc: 0.7559 - val_prc: 0.5084
Epoch 3/100
761/761 [==============================] - 2s 3ms/step - loss: 0.4854 - tp: 5916.0000 - fp: 2834.0000 - tn: 145598.000